In [1]:
import pandas as pd
import os

In [2]:
path_dataset = os.path.join(os.path.dirname(os.getcwd()), 'Dataset')

In [3]:
# remove "title" and "content" from the content
# remove "The user" from the beginning of the summary

df_issues = pd.read_json(os.path.join(path_dataset, 'issues_original.json'))
df_questions = pd.read_json(os.path.join(path_dataset, 'questions_original.json'))

df_issues['Issue_original_content'] = df_issues['Issue_original_content'].apply(
    lambda x: x.replace('Title: ', '').replace('; Content:', ''))
df_issues['Issue_original_content_gpt_summary'] = df_issues['Issue_original_content_gpt_summary'].apply(
    lambda x: x.removeprefix('The user '))
df_issues['Issue_preprocessed_content'] = df_issues['Issue_preprocessed_content'].apply(
    lambda x: x.replace('Title: ', '').replace('; Content:', ''))

df_questions['Question_original_content'] = df_questions['Question_original_content'].apply(
    lambda x: x.replace('Title: ', '').replace('; Content:', ''))
df_questions['Question_original_content_gpt_summary'] = df_questions['Question_original_content_gpt_summary'].apply(
    lambda x: x.removeprefix('The user '))
df_questions['Question_preprocessed_content'] = df_questions['Question_preprocessed_content'].apply(
    lambda x: x.replace('Title: ', '').replace('; Content:', ''))

df_issues['Challenge_original_content'] = df_issues['Issue_original_content']
df_issues['Challenge_original_content_gpt_summary'] = df_issues['Issue_original_content_gpt_summary']
df_issues['Challenge_preprocessed_content'] = df_issues['Issue_preprocessed_content']

df_questions['Challenge_original_content'] = df_questions['Question_original_content']
df_questions['Challenge_original_content_gpt_summary'] = df_questions['Question_original_content_gpt_summary']
df_questions['Challenge_preprocessed_content'] = df_questions['Question_preprocessed_content']

df_questions['Solution_original_content'] = df_questions['Answer_original_content']
df_questions['Solution_original_content_gpt_summary'] = df_questions['Answer_original_content_gpt_summary']
df_questions['Solution_preprocessed_content'] = df_questions['Answer_preprocessed_content']

del df_issues['Issue_original_content']
del df_issues['Issue_original_content_gpt_summary']
del df_issues['Issue_preprocessed_content']

del df_questions['Question_original_content']
del df_questions['Question_original_content_gpt_summary']
del df_questions['Question_preprocessed_content']

del df_questions['Answer_original_content']
del df_questions['Answer_original_content_gpt_summary']
del df_questions['Answer_preprocessed_content']

df_all = pd.concat([df_issues, df_questions], ignore_index=True)
df_all.to_json(os.path.join(path_dataset, 'all_original.json'), indent=4, orient='records')


In [12]:
df_all = pd.read_json(os.path.join(path_dataset, 'all_original.json'))

# remove issues with uninformed content
for index, row in df_all.iterrows():
    if len(row['Challenge_original_content']) < 30:
        df_all.drop(index, inplace=True)
    elif row['Solution_original_content'] and len(row['Solution_original_content']) < 30:
        df_all.drop(index, inplace=True)

df_all.to_json(os.path.join(path_dataset, 'all_filtered.json'), indent=4, orient='records')

In [ ]:
# visualize the best challenge topic model

from sklearn.feature_extraction.text import TfidfVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-mpnet-base-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=10, n_components=2,
                  metric='manhattan', low_memory=False)

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=50, max_cluster_size=1500)

# Step 4 - Tokenize topics
vectorizer_model = TfidfVectorizer(stop_words="english", ngram_range=(1, 4))

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# Step 6 - (Optional) Fine-tune topic representation
representation_model = KeyBERTInspired()

# All steps together
topic_model = BERTopic(
    embedding_model=embedding_model,            # Step 1 - Extract embeddings
    umap_model=umap_model,                      # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,                # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,          # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                  # Step 5 - Extract topic words
    # Step 6 - (Optional) Fine-tune topic represenations
    representation_model=representation_model,
    # verbose=True                              # Step 7 - Track model stages
)

df_all = pd.read_json(os.path.join(path_dataset, 'all_filtered.json'))
docs = df_all['Challenge_original_content_gpt_summary'].tolist()

topic_model = topic_model.fit(docs)
# topic_model.save(os.path.join(path_dataset, 'Topic model'))

# fig = topic_model.visualize_topics()
# fig.write_html(os.path.join(path_dataset, 'Topic visualization.html'))

# fig = topic_model.visualize_barchart()
# fig.write_html(os.path.join(path_dataset, 'Term visualization.html'))

# fig = topic_model.visualize_heatmap()
# fig.write_html(os.path.join(path_dataset, 'Topic similarity visualization.html'))

# fig = topic_model.visualize_term_rank()
# fig.write_html(os.path.join(path_dataset, 'Term score decline visualization.html'))

# hierarchical_topics = topic_model.hierarchical_topics(docs)
# fig = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
# fig.write_html(os.path.join(path_dataset, 'Hierarchical clustering visualization.html'))

# embeddings = embedding_model.encode(docs, show_progress_bar=False)
# fig = topic_model.visualize_documents(docs, embeddings=embeddings)
# fig.write_html(os.path.join(path_dataset, 'Document visualization.html'))

info_df = topic_model.get_topic_info()
info_df


In [ ]:
# visualize the best solution topic model

from sklearn.feature_extraction.text import TfidfVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-mpnet-base-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=10, n_components=2,
                  metric='manhattan', low_memory=False)

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=50, max_cluster_size=1500)

# Step 4 - Tokenize topics
vectorizer_model = TfidfVectorizer(stop_words="english", ngram_range=(1, 4))

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# Step 6 - (Optional) Fine-tune topic representation
representation_model = KeyBERTInspired()

# All steps together
topic_model = BERTopic(
    embedding_model=embedding_model,            # Step 1 - Extract embeddings
    umap_model=umap_model,                      # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,                # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,          # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                  # Step 5 - Extract topic words
    # Step 6 - (Optional) Fine-tune topic represenations
    representation_model=representation_model,
    # verbose=True                              # Step 7 - Track model stages
)

df_all = pd.read_json(os.path.join(path_dataset, 'all_filtered.json'))
docs = df_all[df_all['Challenge_original_content'] != '']['Challenge_original_content_gpt_summary'].tolist()

topic_model = topic_model.fit(docs)
# topic_model.save(os.path.join(path_dataset, 'Topic model'))

# fig = topic_model.visualize_topics()
# fig.write_html(os.path.join(path_dataset, 'Topic visualization.html'))

# fig = topic_model.visualize_barchart()
# fig.write_html(os.path.join(path_dataset, 'Term visualization.html'))

# fig = topic_model.visualize_heatmap()
# fig.write_html(os.path.join(path_dataset, 'Topic similarity visualization.html'))

# fig = topic_model.visualize_term_rank()
# fig.write_html(os.path.join(path_dataset, 'Term score decline visualization.html'))

# hierarchical_topics = topic_model.hierarchical_topics(docs)
# fig = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
# fig.write_html(os.path.join(path_dataset, 'Hierarchical clustering visualization.html'))

# embeddings = embedding_model.encode(docs, show_progress_bar=False)
# fig = topic_model.visualize_documents(docs, embeddings=embeddings)
# fig.write_html(os.path.join(path_dataset, 'Document visualization.html'))

info_df = topic_model.get_topic_info()
info_df
